# 다층 퍼셉트론 모델 구조

## 수학 연산과 각종 부수적 기능

###  파이썬 모듈 불러들이기

In [ ]:
import numpy as np
import time
import os
import csv
import copy
import wave
import cv2
import matplotlib.pyplot as plt

from PIL import Image
from IPython.core.display import HTML

### ReLU 관련 함수 정의

In [ ]:
def relu(x):
    return np.maxium(x, 0)

def relu_derv(y):
    return np.sign(y)

### 시그모이드 관련 함수 정의

In [ ]:
def sigmoid(x):
    return np.exp(-relu(x)) / (1.0 + np.exp(-np.abs(x)))

def sigmoid_derv(y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

### 쌍곡탄젠트 관련 함수 정의

In [ ]:
def tanh(x):
    return 2 * sigmoid(2*x) - 1

def tanh_derv(y):
    return (1.0 + y) * (1.0 - y)

### 소프트맥스 관련 함수 정의

In [ ]:
def softmax(x):
    max_elem = np.max(x, axis = 1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis = 1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs + 1.0e-10), axis = 1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

### csv 파일 읽기

In [ ]:
def load_csv(path, skip_header=True):
    with open(path) as csvfile:
        csvreader = csv.reader(path)
        headers = None
        if skip_header:
            headers = nex(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)

    return rows, headers

### txt 파일 읽기

In [ ]:
def load_txt(path, skip_header=True):
    with open(path, 'r') as txtfile:
        headers = None
        if skip_header:
            headers = txtfile.readline()
        lines = txtfile.readlines()
        lines.rstrip()
        for line in lines:
            row = [value for value in line.split('\t')]
            rows.append(row)

    return rows, headers

### 벡터 처리 지원 함수의 정의

In [ ]:
def onehot(xs, cnt):
    return np.eye(cnt)[np.array(xs).astype(int)]

def vector_to_str(x, fmt='%.2f', max_cnt=0):
    if max_cnt == 0 or len(x) <= max_cnt:
        return '[' + ','.join([fmt] * len(x)) % tuple(x) + ']'
    v = x[0:max_cnt]
    return '[' + ','.join([fmt] * len(v)) % tuple(x) + ',...]'

### 폴더 정보 읽기 함수의 정의

In [ ]:
def list_dir(path):
    filenames = os.listdir(path)
    filenames.sort()
    return filenames